# Company Brochure Generator UI
Generates a brochure for a company website, after scraping the website and pages linked with that page, based on the provided company URL. 
Enables users to 
- Choose a model type (Llama 3.2, Claude, GPT)-
- Choose the tone preference
- Choose the target audience

#### Imports, Keys, Instantiation

In [38]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import anthropic
import gradio as gr

In [15]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [20]:
# Define models
CLAUDE_MODEL = "claude-3-haiku-20240307"
GPT_MODEL = "gpt-4o-mini"

In [17]:
# Creating instances
claude = anthropic.Anthropic()
openai = OpenAI()

#### Web Scraper

In [8]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email or social media links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [36]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        print(f"Processing {link['url']}...")
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

#### System Message

In [18]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. \
You are also provided with the tone, and the target audience. Provide an appropriate answer. Respond in markdown."

#### LLM Call Functions

In [40]:
def stream_gpt(prompt,company_name,url):
    messages = [
        {"role": "user", "content": prompt},
        {"role":"system","content":system_message}
    ]
    stream = openai.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [21]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

#### Brochure Creation

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
def create_brochure(company_name, url,model,tone,target):
    print('create brochure function called')
    prompt = f"Please generate a company brochure for {company_name}."
    prompt += f"Use a {tone} tone; and target content at {target}"
    prompt += get_brochure_user_prompt(company_name,url)
    
    if model == "GPT":
        result = stream_gpt(prompt,company_name,url)
    elif model=="Claude":
        result = stream_claude(prompt,company_name,url)
    else:
        raise ValueError("Unknown model")
    yield from result

#### Putting it all together : Gradio UI

In [22]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [41]:
gr.Interface(
    fn=create_brochure,
    inputs=[
        gr.Textbox(label='Company Name:'),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(['GPT','Claude'],label='Select Model:'),
        gr.Dropdown(['Formal','Casual','Persuasive','Informative','Conversational'],label='Select Tone:'),
        gr.Dropdown(['Businesses','General Public','Students','Investors','Customers'],label='Select Target Audience:'),
    ],
    outputs = [gr.Markdown(label='Brochure')],
    flagging_mode = 'never',
    js = force_dark_mode
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://www.vellum.ai/'}, {'type': 'careers page', 'url': 'https://www.vellum.ai/careers'}]}
Processing https://www.vellum.ai/...
Processing https://www.vellum.ai/careers...
